<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/model1_cross_validation_provaPrecisione.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
@File name: model1_cross_validation.ipynb
@Created on 2018-12-20
@Authors: Federica Gerina, Francesca Moi, Silvia Maria Massa
@Description: Given a time-series dataset that contains minute-by-minute data 
about different kind of gases, collected by the uHoo air quality sensor, train
a NN that classifies if a minute belongs to the class "Pasto" (1) otherwise to
the class "Other" (0).
'''

!pip install liac-arff

import arff
import math

import numpy as np

from keras import optimizers
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, LeakyReLU, BatchNormalization, Activation
from keras.callbacks import EarlyStopping

from sklearn.utils import compute_class_weight
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

import sys
sys.path.append('local_modules')

import postprocessing_sw

Using TensorFlow backend.


In [0]:
#fix random seed for reproducibility
seed = 5
np.random.seed(seed)

In [0]:
'''
@Description: generate a multilayer perceptron with LeakyRelu as activation
function.
@param: 
  - shape : int, the shape of the input
  - n_features: int, the number of features given
'''

def generate_model_leaky(shape, n_features):
  
  units_1 = int(n_features/2)
  units_2 = n_features
  units_3 = n_features*2
  
  model = Sequential()
  model.add(BatchNormalization())
  
  model.add(Dense(units_1, input_dim=shape, kernel_initializer='random_uniform',  use_bias = False))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.5))
  
  model.add(Dense(units_2, kernel_initializer='random_uniform',  use_bias = False))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.5))
  
  model.add(Dense(units_3, kernel_initializer='random_uniform',  use_bias = False))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.5))
  
  model.add(Dense(1, activation='sigmoid'))
  #print(model.summary())

  return model

In [4]:
#@title CHOOSE

'''
@Description: MAIN
'''

#LOAD DATA
print("Loading data...")

dataset = '/root/data/uHooComplete_featureDataset_Past_Only.arff' #@param {type:"string"}

with open (dataset, encoding='utf-8') as f:
  dataDictionary = arff.load(f)

data = np.array(dataDictionary['data'])
print("DATASET LOADED")

#CONVERTING VALUES
print("\nConverting values...")
for i in data:
  if(i[-1] == 'Other'): i[-1] = 0
  elif(i[-1] == 'Pasto') : i[-1] = 1

dataset = data.astype('float32')
print("CONVERSION DONE")

#SPLIT INTO INPUT (X) AND OUTPUT (Y) VARIABLES
s = dataset.shape[-1]
X = dataset[:,0:s-1]
Y = dataset[:,s-1]

n_features = s-1

#OPTIMIZERS
adm = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

#LOSS
loss = 'binary_crossentropy'

#DEFINE K-FOLD CROSS-VALIDATION
fold = 10 #@param {type:"integer"}

kfold = KFold(n_splits=fold, shuffle=False, random_state=None)

cvscores = []
predictions = []
true_p =[]
true_n = []
precision = []
test_time =[]
real = []

dimSplit = math.floor(len(dataset[:,0])/10)
startIndex = 0
finishIndex = dimSplit-1

i = 1

for train, test in kfold.split(X, Y):
  
  print("\nFOLD: %d" %i)
  
  #COMPUTE CLASS WEIGHT
  labels = np.unique(Y[train])
  classWeight = compute_class_weight('balanced', labels, Y[train])
  classWeight = dict(zip(labels,classWeight))

  #GENERATE MODEL
  print("\nGenerate model...")

  model = generate_model_leaky(X[train].shape[-1], n_features)

  #COMPILE MODEL
  print("\nCompile model...")
  model.compile(loss = loss, optimizer = adm , metrics=['accuracy'])

  #FIT MODEL
  print("\nFit model...")
  epochs = 10 #@param{type:"integer"}
  history = model.fit(X[train], Y[train], epochs=epochs, batch_size = 128, shuffle = True, verbose=1, class_weight = classWeight)

  #EVALUATE MODEL
  print("\nEvaluate model...")
  scores_test = model.evaluate(X[test], Y[test], batch_size= 128, verbose = 0)
  print("Test loss: %.2f%%" % (scores_test[0] * 100))
  print("Test accuracy: %.2f%%" % (scores_test[1] * 100))
  
  cvscores.append(scores_test[1] * 100)
  
  #CALCULATE PREDICTIONS
  print("\nCalculate predictions...")
  pred = model.predict_classes(X[test], batch_size=128, verbose=0)
  flat_pred = [item for sublist in pred for item in sublist]
  predictions.append(flat_pred)
  real.append(Y[test])
  
  #STORE DATETIME
  time = []  
  for j in X[test]:
    time.append(int(j[-5]))  
  test_time.append(time)
  
  #CONFUSION MATRIX
  print("\nCompute confusion matrix...")
  y_true = Y[test]
  tn, fp, fn, tp = confusion_matrix(y_true, pred).ravel()
  other = 100*tn/(tn+fp)
  pasto = 100*tp/(fn+tp)
  prec = 100*(tp/(tp+fp))
  true_p.append(pasto)
  true_n.append(other)
  precision.append(prec)
  print("Other: %.2f %%" % other)
  print("Pasto: %.2f %%" % pasto)
  print("Precision: %.2f %%" % prec)
  
  i+=1

print("MEAN ACCURACY: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("MEAN TRUE POSITIVE RATE: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(true_p), np.std(true_p)))
print("MEAN TRUE NEGATIVE RATE: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(true_n), np.std(true_n)))
print("MEAN PRECISION: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(precision), np.std(precision)))

#FLATTENING LISTS
flat_predictions = [item for sublist in predictions for item in sublist]
true_y = [arr.tolist() for arr in real]
flat_true_y = [item for sublist in true_y for item in sublist]
flat_time = [item for sublist in test_time for item in sublist]


Loading data...
DATASET LOADED

Converting values...
CONVERSION DONE
Instructions for updating:
Colocations handled automatically by placer.

FOLD: 1

Generate model...

Compile model...

Fit model...
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
315441/315441 [==============================] - 10s 33us/step - loss: 0.6358 - acc: 0.6932
Epoch 2/10
315441/315441 [==============================] - 9s 29us/step - loss: 0.5392 - acc: 0.7532
Epoch 3/10
315441/315441 [==============================] - 9s 27us/step - loss: 0.5111 - acc: 0.7589
Epoch 4/10
315441/315441 [==============================] - 9s 28us/step - loss: 0.4932 - acc: 0.7622
Epoch 5/10
315441/315441 [==============================] - 9s 27us/step - loss: 0.4838 - acc: 0.7656
Epoch 6/10
315441/315441 [==============================] - 9s 27us/step - loss: 0.4740 - acc: 0.7687
Epoch 7/10
315441/31544

In [17]:
#POST PROCESSING
new_pred = postprocessing_sw.sliding_windows(flat_predictions,91)

#CONFUSION MATRIX BEFORE POST PROCESSING
print("\nCompute confusion matrix BEFORE POST PROCESSING...")
tn, fp, fn, tp = confusion_matrix(flat_true_y, flat_predictions).ravel()
print("TN", tn)
print("FP", fp)
print("FN", fn)
print("TP", tp)
other = 100*tn/(tn+fp)
pasto = 100*tp/(fn+tp)
precision = 100*(tp/(tp+fp))
print("Other corretti: %.2f %%" % other)
print("Pasto corretti: %.2f %%" % pasto)
print("Precision: %.2f %%" % precision)

#CONFUSION MATRIX AFTER POST PROCESSING
print("\nCompute confusion matrix AFTER POST PROCESSING...")
tn, fp, fn, tp = confusion_matrix(flat_true_y, new_pred).ravel()
print("TN", tn)
print("FP", fp)
print("FN", fn)
print("TP", tp)
other = 100*tn/(tn+fp)
pasto = 100*tp/(fn+tp)
precision = 100*(tp/(tp+fp))
print("Other corretti: %.2f %%" % other)
print("Pasto corretti: %.2f %%" % pasto)
print("Precision: %.2f %%" % precision)


SLIDING WINDOWS FUNCTION...

Compute confusion matrix BEFORE POST PROCESSING...
TN 282037
FP 52130
FN 4980
TP 11344
Other corretti: 84.40 %
Pasto corretti: 69.49 %
Precision: 17.87 %

Compute confusion matrix AFTER POST PROCESSING...
TN 292646
FP 41521
FN 5884
TP 10440
Other corretti: 87.57 %
Pasto corretti: 63.95 %
Precision: 20.09 %


In [0]:
def get_precision(p, r):
  index_pred = []
  index_real = []

  prediction_meals = []
  real_meals = []

  #p = flat_pred
  #r = test_label

  #Costruzione delle liste di indici: una di pasti predetti e una di pasti reali

  for i in range(0, len(p)):
    if p[i] == 1:
      index_pred.append(i)
      if ((i+1) in range(0, len(p)) and p[i+1] == 0) or (i+1) not in range(0, len(p)): 
        if(len(index_pred)>5):        
          prediction_meals.append(index_pred)
        index_pred = []
    if r[i] == 1:
      index_real.append(i)
      if ((i+1) in range(0, len(p)) and r[i+1] == 0) or (i+1) not in range(0, len(p)): 
        real_meals.append(index_real)
        index_real = []

  #Ricerca delle intersezioni

  intersection = []
  tp = 0
  fp = 0

  for i in range(0, len(real_meals)): #inizializzazione
    intersection.append(0)

  for i in range(0, len(prediction_meals)): #per ogni pasto predetto
    flag_found = 0 #per tenere conto se l'intersezione è già stata trovata all'interno del pasto reale

    j=0
    while j in range(0, len(real_meals)) and flag_found==0: # per ogni pasto reale    
      #se almeno un elemento di prediction_meals[i] è presente in j
      flag_visited = 0 

      for x in prediction_meals[i]: #da qui potrei mettere una funz

        if x in real_meals[j]: #se c'è l'intersezione        

          if intersection[j]==0: #controlla che non sia già stata trovata          
            intersection[j] = 1
            flag_visited = 1 #imposta il flag a uno per indicare che il vero positivo è stato già individuato e per non contarlo più di una volta
            flag_found = 1 #imposta il flag a uno per indicare che il pasto predetto è stato trovato nei pasti reali
            tp=tp+1

          elif flag_visited == 0: #se è stata già trovata per un altro pasto e quindi è la prima volta che controllo questo pasto predetto,
            fp=fp+1
            flag_visited = 1 #imposta il flag a uno per indicare che il falso positivo è stato già individuato e per non contarlo più di una volta
            flag_found = 1

      j = j + 1  
    #end for j

    if flag_found == 0: #se l'intersezione nn è stata trovata nei pasti reali il pasto predetto è falso
      fp=fp+1
      #print("FP3")

  #Calcolo accuratezza
  print("\nPrecision: ")
  print("N° pasti reali:", len(real_meals))
  print("N° pasti predetti:", len(prediction_meals))

  print("TP:", tp)
  print("FP:", fp)

  somma = tp+fp
  precision = (tp/somma)*100
  TPR = (tp/len(real_meals))*100

  print("Tot = ", somma)
  print("TPR = ", TPR)
  print("Precisione: %.2f %%" % precision)



In [19]:
get_precision(flat_predictions,flat_true_y)



Precision: 
N° pasti reali: 669
N° pasti predetti: 2044
TP: 454
FP: 1590
Tot =  2044
TPR =  67.86248131539611
Precisione: 22.21 %


In [23]:
get_precision(new_pred, flat_true_y)


Precision: 
N° pasti reali: 669
N° pasti predetti: 509
TP: 314
FP: 195
Tot =  509
TPR =  46.93572496263079
Precisione: 61.69 %


In [0]:
def getMeals(pred, real):
  index=[]
  index_pred = []
  index_real = []

  prediction_meals = []
  real_meals = []

  ''' Costruzione delle liste: una di pasti predetti e una di pasti reali '''

  #Costruzione pasti predetti

  for i in range(0, len(pred)):

    if pred[i] == 0:  

      index.append(i)

      if ((i+1) in range(0, len(pred)) and pred[i+1] == 1) or (i+1) not in range(0, len(pred)): 

        index_pred.append(index)
        prediction_meals.append(0)
        index = []

    if pred[i] == 1:

      index.append(i)

      if ((i+1) in range(0, len(pred)) and pred[i+1] == 0) or (i+1) not in range(0, len(pred)): 

        index_pred.append(index)
        prediction_meals.append(1)
        index = []

  #Costruzione pasti reali  

  for i in range(0, len(real)):

    if real[i] == 0:  

      index.append(i)

      if ((i+1) in range(0, len(real)) and real[i+1] == 1) or (i+1) not in range(0, len(real)): 

        index_real.append(index)
        real_meals.append(0)
        index = []

    if real[i] == 1:

      index.append(i)

      if ((i+1) in range(0, len(real)) and real[i+1] == 0) or (i+1) not in range(0, len(real)): 

        index_real.append(index)
        real_meals.append(1)
        index = []

  return real_meals

In [24]:
real_meals = getMeals(new_pred, flat_true_y)
print(len(real_meals))

1339
